In [6]:
import json

import pvl
import requests

## Create the necessary input data

To create an ISD we require the following:

- target_name, e.g. Mercury
- capture_date (PDS data product dates appear to be ISO 8601 compliant)
- instrument, e.g. mdis-nac
- focal_plane_temperature (optionally included for temperature dependent focal lengths)
- spacecraft_id, e.g. messenger
- spacecraft_clock_count, from the label
- exposure_duration, from the label
- lighttime_correction, to tell spice what to use
- min_elevation (make optional?)
- max_elevation (make optional?)

For the last two, it might be easier/better to take the intital center point from the PDS label and hit a low res. DEM to find these automatically for the user.

In [2]:
label = pvl.load('/data/messenger_sample/EN0211587012M.IMG')
data = {'target_name': label['TARGET_NAME'],
        'capture_date': label['START_TIME'].isoformat(),
        'instrument': label['INSTRUMENT_NAME'], 
        'focal_plane_temperature': label['FOCAL_PLANE_TEMPERATURE'].value,
        'spacecraft_id' : label['INSTRUMENT_HOST_NAME'],
        'spacecraft_clock_count': label['SPACECRAFT_CLOCK_START_COUNT'],
        'exposure_duration': label['EXPOSURE_DURATION'].value,
        'lighttime_correction':'LT+S', 
        'min_elevation': -100,
        'max_elevation': 100
        }
import json
data = json.dumps(data)

## Standard API landing page

In [3]:
r = requests.get('http://localhost:5000')
r.json()

{'apis': {'1.0': 'api/1.0'}, 'status': 'success'}

## Pushing the API into a namespace and versioning is good practice

In [4]:
r = requests.get('http://localhost:5000/api/1.0')
r.json()

{'data': {'available_missions': {'mercury': {'messenger': '/api/1.0/mercury/messenger'}}},
 'success': True}

## Example listing of the kernels

In [5]:
r = requests.get('http://localhost:5000/api/1.0/mercury/messenger/available_kernels')
r.json()

{'data': {'description': 'All available meta kernels for a given body and mission in sorted order.  The first meta kernel in the list will be loaded unless a different metakernel is specified.',
  'kernels': {'2004': ['/data/spice/mess-e_v_h-spice-6-v1.0/messsp_1000/extras/mk/msgr_2004_v13.tm',
    '/data/spice/mess-e_v_h-spice-6-v1.0/messsp_1000/extras/mk/msgr_2004_v12.tm',
    '/data/spice/mess-e_v_h-spice-6-v1.0/messsp_1000/extras/mk/msgr_2004_v11.tm',
    '/data/spice/mess-e_v_h-spice-6-v1.0/messsp_1000/extras/mk/msgr_2004_v10.tm',
    '/data/spice/mess-e_v_h-spice-6-v1.0/messsp_1000/extras/mk/msgr_2004_v09.tm',
    '/data/spice/mess-e_v_h-spice-6-v1.0/messsp_1000/extras/mk/msgr_2004_v08.tm',
    '/data/spice/mess-e_v_h-spice-6-v1.0/messsp_1000/extras/mk/msgr_2004_v07.tm',
    '/data/spice/mess-e_v_h-spice-6-v1.0/messsp_1000/extras/mk/msgr_2004_v06.tm',
    '/data/spice/mess-e_v_h-spice-6-v1.0/messsp_1000/extras/mk/msgr_2004_v05.tm',
    '/data/spice/mess-e_v_h-spice-6-v1.0/messsp_

## Example generation of the ISD object

Notice that every endpoint returns with a `success` key and a `data` key at the top level.  This helps the user know if the call was successful and that all of the response is within the `data` key.

In [23]:
r = requests.get('http://localhost:5000/api/1.0/csm_isd')
r.json()

{'data': 'Describe this end point', 'success': True}

In [22]:
r = requests.post('http://localhost:5000/api/1.0/csm_isd', data=data)
r.json()

{'data': {'isd': {'boresight': [0.0, 0.0, 1.0],
   'ccd_center': [512.5, 512.5],
   'ephemeris_time': 423895046.5997792,
   'focal_length': 78.2367,
   'focal_length_epsilon': 0.15,
   'ifov': 179.6,
   'instrument_id': 'MSGR_MDIS_WAC',
   'itrans_line': [0.0, -0.0006761060916520159, 71.4399371332522],
   'itrans_sample': [0.0, 71.42857143, 0.0],
   'kappa': -2.9480471764203147,
   'max_elevation': 100,
   'min_elevation': -100,
   'model_name': 'ISIS_MDISNAC_USGSAstro_1_Linux64_csm30.so',
   'nlines': 1024,
   'nsamples': 1024,
   'odt_x': [0.0,
    1.0,
    0.0,
    -7.720894252056575e-05,
    3.599871902138938e-06,
    0.0,
    5.509035727272325e-06,
    0.0,
    5.509035727272406e-06,
    0.0],
   'odt_y': [0.0,
    0.0,
    1.000000000026148,
    0.0,
    -7.720894252092194e-05,
    3.599871782473616e-06,
    0.0,
    5.509035621941527e-06,
    0.0,
    5.5090308738198125e-06],
   'omega': 2.901663189203809,
   'original_half_lines': 512.0,
   'original_half_samples': 512.0,
   'p

## Use the ISD

In [7]:
import usgscam as cam
from cycsm.isd import Isd

In [8]:
isd = r.json()['data']['isd']

In [9]:
i = Isd.loads(isd)

In [10]:
plugin = cam.mdis.MdisPlugin()
camera = plugin.from_isd(i, plugin.modelname(0))

In [11]:
camera.imagesize

(1024.0, 1024.0)

In [12]:
xyz = camera.imageToGround(512.5, 512.5, 0)
xyz

[-1108876.4666681145, -2124897.510113989, -453735.5101609463]

In [13]:
camera.groundToImage(*xyz)

[512.5000000000206, 512.5000000000109]

## Random Messenger Image

The above could be perceived as being too canned.  Here, pull a random image from the PDS and instantiate a camera model.

In [1]:
from ftplib import FTP
import os
from random import choice

In [2]:
# Run only once to avoid too many open FTP connections
ftp = FTP('pdsimage2.wr.usgs.gov')
ftp.login()
dirs = ftp.nlst('archive/a15c-l-mc-2-scanned-images-v1.0/A15MC_0001/DATA')

In [22]:
data_dir = choice(dirs)
# Omit the tif - no need for image data
flist = ftp.nlst(data_dir)
flist = [i for i in flist if os.path.splitext(i)[1] == '.lbl']
f = choice(flist)
fname = os.path.basename(f)
with open('/data/apollo_sample/{}'.format(fname), 'wb') as file:
    ftp.retrbinary('RETR {}'.format(f), file.write)

In [23]:
newfile = os.path.join('/data/apollo_sample/{}'.format(fname))
# Fix the PDS label
import fileinput
for line in fileinput.input(newfile, inplace=True, backup='.bak'):
    if 'HEADER_TYPE' in line and '=' in line:
        print('HEADER_TYPE = "N/A"',)
    else:
        print(line.rstrip())

In [49]:
# Simple helper function to use the API
def create_data_pacakge(in_file):
    label = pvl.load(in_file)
    data = {'target_name': label['TARGET_NAME'],
            'capture_date': label['START_TIME'].isoformat(),
            'instrument': label['INSTRUMENT_NAME'], 
            'spacecraft_id' : label['INSTRUMENT_HOST_NAME'],
            'spacecraft_clock_count': label['SPACECRAFT_CLOCK_START_COUNT'],
            'lighttime_correction':'LT+S', 
            'min_elevation': -100,
            'max_elevation': 100
            }
    return json.dumps(data)

print('Processing a new image: {}'.format(newfile))

# Call the func to create the data package for submission to the micro-service
data = create_data_pacakge(newfile)
print(data)
# Call the micro-service
r = requests.post('http://localhost:5000/api/1.0/csm_isd', data=data)

# Get the ISD back and instantiate a local ISD for the image
isd = r.json()['data']['isd']
i = Isd.loads(isd)

# Create the plugin and camera as usual
plugin = cam.mdis.MdisPlugin()
camera = plugin.from_isd(i, plugin.modelname(0))

# Call I2G
xyz = camera.imageToGround(512.5, 512.5, 0)
print(xyz)

# Call G2I
camera.groundToImage(*xyz)

Processing a new image: /data/apollo_sample/AS15-M-1735.lbl
{"target_name": "MOON", "capture_date": "1971-08-02T08:45:57.251000", "instrument": "METRIC CAMERA", "spacecraft_id": "APOLLO 15 COMMAND AND SERVICE MODULE", "spacecraft_clock_count": "UNK", "lighttime_correction": "LT+S", "min_elevation": -100, "max_elevation": 100}


JSONDecodeError: Expecting value: line 1 column 1 (char 0)